In [ ]:
!python --version

In [ ]:
!echo $CONDA_DEFAULT_ENV

## 🧪 O que é um Teste de Hipótese?

É um método estatístico usado para tomar decisões se populações ou amostras são similares ou não, avaliando se os dados fornecem evidência suficiente para rejeitar uma suposição inicial.

### 🧩 Componentes essenciais

#### 1️⃣ Hipóteses

  1. Hipótese nula (H₀)
    - Afirmação padrão, status quo, 'distribuições semilares'
      - Exemplo:
        - As médias e desvio padrão são similares (caso x controle)
        - As médias e desvio padrão são similares para os três grupos ~Normal (ANOVA)
        - Pacientes controle e tratados são semilares
        - Pacientes após dieta tem distribuição de peso similar a antes da dieta.
        - O medicamento novo tem efeito similar ao antigo (controle)

  2. Hipótese alternativa (H₁ ou Hₐ)
    - Afirmação contrária, rejeitar a Hipótese Nula (H₀)
      - Exemplo:
        - As médias e desvio padrão são diferentes (caso x controle) - mas, quanto? há efeito?
        - As médias e desvio padrão são diferentes para os três grupos ~Normal (ANOVA)
          - Mas quais grupos e quanto
        - Pacientes controle tem resultados distintos dos tratados
        - Pacientes após dieta tem média de peso inferior a antes da dieta.
        - O medicamento novo resulta em menos efeitos colaterais comparados ao medicamento antigo (controle)

### Teste de Hipótese paramétricos - testes e distribuições

  - 2 distribuições Normais: teste Z
  - Teste de Normalidade (Shapiro-Wilk)
  - 2 distribuições Normais com baixo número amostral (N<30):
    - t-student: pareado e não pareado
  - Mais que 2 grupos com distribuição Normal e desvio padrão similar: ANOVA
  - Não tenho ou não confio em meu desvio padrão!
    - O que fazer? Bootstrap
  - Distribuições normais, mais 2 grupos, todos contra todos: teste de Tukey
  - Distribuições normais, mais 2 grupos ou mais versus uma referência: teste de Dunnett
  - Tabelas de contingência - variáveis nominais e valores discretos: chi-quadrado (χ²)
    - A distriuição desta tabela é similar à distribuição esperada?
  - Duas distribuições são similares:
    - Goodness of fit
    - Kolmogorov-Smirnov


### O que podemos aprender além do teste de hipótese? nas próximas aulas

  - revisitando:
    - p-valor (p-value)
    - tamanho de efeito (ES - effect size)
  - o que é confiança?
    - intervalo de confiança
    - barplot com barras de erros
  - Tabela de contingência H₀ x Hₐ
    - True Positive e Negative (TP e TN)
    - False Positive e Negative (FP e FN)
    - Decorrência:
      - Sensibilidade e Especificida (introdução)
  - Poder estatístico
  - Como calcular o n amostral mínimo

#### Biblioteca scipy.stats

  - https://docs.scipy.org/doc/scipy/reference/stats.html
  

### Uma distribuição é diferente da outra?
  - Medicina:
    - Grupo de doentes (diabéticos):
      - mede-se um parâmetro: glicemia (em jejum)
      - medica todos os pacientes
      - espera-se 24h, 48h, 7d, 30d
      - mede-se o efetio da droga sobre a glicemia
    - Modelo experimental - não tão trival - área de conhecimento:
      - controlar os grupos (critérios de inclusão)
        - somente homens de 30 a 45 anos
        - somente caucasianos, orientas, afro-descentes
        - ...
  - Biologia molecular:
    - Uma cultura de células:
      - Controle: PBS (phetal bovine serum)
      - Administrar um Princípio Ativo (droga) ou extrato de veneno (diluído)
      - Esperar 24 h (ou uma série temporal)
      - Medir citocinas inflamatórias, antiinflmatórias e relativas a apoptose.
      - Desejas saber:
        - Os dois grupos (controle e 24 h) são diferentes (para cada marcador)
        - Os dois grupos são diferentes para cada tempo de série temporal (para cada marcador)
        - N+1 grupos (controle e N concentrações) são diferentes (para cada marcador)
      
### Vários grupos

  - Premissas para ANOVA:
     - Número amostral possivelmente grande
     - Distribuição é normal
     - Variância similar entre grupos
  - ANOVA
    - H0 - todos os grupos são similares
    - Ha - ao menos um grupo é estatisticamente diferente
    - Mas qual grupo?
      - todos contra todos: Tukey test
      - todos contra um controle: Dunnett test

In [ ]:
import os, sys, math
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

import matplotlib.pyplot as plt # matplotlib e seu alias plt
from matplotlib import gridspec
%matplotlib inline

### 🔄 Como usar o teste de hipótese

#### Definir as Hipótese:

  - Hipótese nula: H₀
    - A média de peso é igual após a dieta
  - Hipóstese alternativa H₁
    - A média de peso é diferente após a dieta

#### Fluxo / raciocínio:

  - Analisar as variáveis medidas
  - Escolher o teste estatístico adequado às variáveis e grupos analisados
  - Fixar α (alfa) == nível de significância:
    - Muito exigente α = 0.01 (1% da área da distribuição)
    - 'Padrão' α = 0.05 (5% da área da distribuição)
    - Pouco exigente α = 0.10 (10% da área da distribuição)
  - CUIDADO! Bicaudal?
    - Bicaudal - α/2 para cada lado (posso ganhar ou perder peso!)
    - Moncaudal - α para um dos lados
  - Calcular a estatística de teste
    - qual a distância o início ou do centro da distribuição
    - estatística é um número que define a área coberta ou após o corte
    - dada um estatística podemos calcular o p-valor
  - Tomar a decisão (MAS CUIDADO):
    - Dizemos que se a estatística (p-valor) for além α == nível de significância:
      - Temos que rejeitar H₀ e aceitar H₁
      - MAS !!!
        - Temos que sempre analisar o Tamanho de Efeito (ES)
        - Analisar quanto ao experimento se a física/química/biologia aceitam rejeitar H₀
    - Se o p-valor estiver aquém de α == nível de significância:
      - Temos que aceitar H₀
      - Para as condiçoes pré-definidas não há efeito significante
  - NÃO podemos mudara α (nível de significância) depois de se obter este resultado.

### 🔄 Exemplo 1 - Tabela 1 de um Estudo Clínico

Num experimento randomizado espera-se que as características dos indíviduos entre os grupos sejam similaras, logo todos os testes tem que aceitar a hipótese nula

| Variável | Grupo Controle  | Grupo Tratado  | p-valor |  Aceita H₀ |
|----------|-----------------|----------------|---------|------------|
| Gênero |    |    |   |    |
| Homens |  327   |  345  |   |    |
| Mulheres | 333   |  360  | >> .05  |  Sim  |
| Obesidade |     |    | >> .05  |  Sim  |
| Idosos |     |    |  >> .05  |  Sim  |
| Hipertensos |     |  |  >> .05  |  Sim  |



### 🔄 Exemplo 2 - Dieta

Esperimento pareado:  60 homens e 57 mulheres aceitaram participar de um estudo de Dieta baseada em aplicações de Ozempic. Mediu-se o peso de dos participantes antes da dieta, e 90 dias após. Definiu-se um protocolo para que os pacientes se alimentassem como vinham fazendo, e não fizesse nenhuma dieta adicional em nenhuma estravagância alimentar durate o período.

| Variável | Antes  | Depois  | p-valor |  Aceita H₀ |
|----------|-----------------|----------------|---------|------------|
| Peso |    |    |   |    |
| Homens |  73.4 (6.1)   |  67.3 (5.8)  |  ?  |  ?   |
| Mulheres | 66.8 (5.8)   |  64.2 (5.0)  |  ?   |  ?   |

  - Perguntas
    - Qual o teste estátístico?
    - Homens emagresseram?
    - Mulheres emagrresseram?
    - Homens emagressram mais que mulhares?


#### Experimento da Dieta

  - 2 grupos - antes e depois as mesmas pessoas
  - variáveis contínuas
  - teste t-pareado

Antes e depois da dieta

  - homens N 60
  - mulheres N 57

  - homens antes 73.4 (6.1)   depois 67.3 (5.8)
  - mulhares antes  66.8 (5.8) depois 64.2 (5.0)


#### Grupos (e teste) dependentes ou independentes?

  - Dependente: o mesmo grupo antes e depois --> t-test pareado
  - Independente: grupos distintos antes e depois --> t-test não pareado

In [ ]:
import numpy as np
import math
from scipy import stats

#### Comparando-se 2 grupos dependetes

In [ ]:
# unidade = Kg

def simular_grupo_pareado(mu_antes, sd_antes,
                          mu_depois, sd_depois,
                          N, rho=0.7, seed=None):

    if seed is not None:
        np.random.seed(seed)

    antes_lista = np.random.normal(mu_antes, sd_antes, N)
    erro_lista  = np.random.normal(0, 1, N)

    depois_lista = (
        mu_depois +
        # correção da relação de SD e a difrença alcançada antes
        rho * (sd_depois / sd_antes) * (antes_lista - mu_antes) +
        # adicionar ruído proporcional à falta de correlação e SD depois
        np.sqrt(1 - rho**2) * sd_depois * erro_lista
    )

    return antes_lista, depois_lista


In [ ]:
#---------- dados homens ------------
N_hom = 60

peso_ctrl_hom = 73.4
peso_diet_hom = 67.3
SSD_ctrl_hom = 6.1
SSD_diet_hom = 5.8


In [ ]:
#---------- dados mulheres ------------
N_mul = 57

peso_ctrl_mul = 66.8
peso_diet_mul = 64.2
SSD_ctrl_mul = 5.8
SSD_diet_mul = 5.0


In [ ]:
def calc_show_ttest_dependente(genero, N, lista_depois, lista_antes, alpha=0.05):

    lista_diff = lista_depois - lista_antes
    
    # t-test relacionados
    # This is a test for the null hypothesis that two related or repeated samples have identical average (expected) values
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html
    t, pval = stats.ttest_rel(lista_depois, lista_antes)

    # ddof=1 --> divide por N-1
    SSDpool = lista_diff.std(ddof=1)
    diff = lista_diff.mean()
    ES = diff / SSDpool

    msg = f"{N} {genero} usando teste-t pareado = estatística {t:.3f} e p-valor {pval:.2e}"
    
    if pval >= alpha:
        msg += '\n' + "Aceitamos a Hipótese nula, não houve efeito com a dieta"
    else:
        msg += '\n' + "Rejeitamos a Hipótese nula, houve efeito com a dieta"
    
    msg += '\n' + f"Tamanho do efeito para {N} {genero} = {ES:.1f} com Diferença={diff:.1f} Kg e SSD_pool = {SSDpool:.1f} Kg"

    return t, pval, ES, SSDpool, msg

In [ ]:
lista_ctrl_hom, lista_diet_hom = simular_grupo_pareado(
    mu_antes=peso_ctrl_hom, sd_antes=SSD_ctrl_hom,
    mu_depois=peso_diet_hom, sd_depois=SSD_diet_hom,
    N=N_hom, rho=0.7, seed=None)

t_hom, pval_hom, ES_hom, SSD_pool_hom, msg_hom = \
calc_show_ttest_dependente('homens', N_hom, lista_diet_hom, lista_ctrl_hom)

print(msg_hom)

In [ ]:
lista_ctrl_mul, lista_diet_mul = simular_grupo_pareado(
    mu_antes=peso_ctrl_mul, sd_antes=SSD_ctrl_mul,
    mu_depois=peso_diet_mul, sd_depois=SSD_diet_mul,
    N=N_mul, rho=0.7, seed=None)

t_mul, pval_mul, ES_mul, SSD_pool_mul, msg_mul = \
calc_show_ttest_dependente('mulheres', N_mul, lista_diet_mul, lista_ctrl_mul)

print(msg_mul)

#### Comparando-se 2 grupos independetes

O que você fez (e está correto)

Simulou grupos independentes ✔️

Calculou SD pooled ✔️

Calculou Cohen’s d para grupos independentes ✔️

Isso corresponde distância de Cohen se distribuições independentes:

<p style="font-size: 24px;">
$ES = \frac{<x_{dieta}>-<x_{controle}>}{SD_{pooled}}$ </p>


In [ ]:
def simular_um_grupo_ind(mu:float, SSD:float, N:int, seed=None):

    if seed is not None and isinstance(seed, int):
        np.random.seed(seed)

    peso_lista = np.random.normal(mu, SSD, N)

    return peso_lista

def calc_SSD_pool(SSD1:float, N1:int, SSD2:float, N2:int) -> float:
    SDpool = math.sqrt( ((N1-1)*SSD1**2 + (N2-1)*SSD2**2) / (N1+N2-2) )
    return SDpool

def calc_show_ttest_ind(genero, N, lista_depois, lista_antes, ES, diff, SSD_pool, alpha=0.05):

    # t-test independente
    t, pval = stats.ttest_ind(
        lista_depois,
        lista_antes,
        equal_var=True   # SD pooled → teste t clássico
    )

    msg = f"{N} {genero} usando teste-t não pareado = estatística {t:.3f} e p-valor {pval:.2e}"
    
    if pval >= alpha:
        msg += '\n' + "Aceitamos a Hipótese nula, não houve efeito com a dieta"
    else:
        msg += '\n' + "Rejeitamos a Hipótese nula, houve efeito com a dieta"
    
    msg += '\n' + f"Tamanho do efeito para {N} {genero} é {ES:.1f} com Diferença={diff:.1f} Kg e SSDpool={SSD_pool:.1f} Kg"

    return t, pval, msg

In [ ]:
## grupos de homens independentes
lista_ctrl_hom = simular_um_grupo_ind(peso_ctrl_hom, SSD_ctrl_hom, N_hom)
lista_diet_hom = simular_um_grupo_ind(peso_diet_hom, SSD_diet_hom, N_hom)

SSD_pool_hom = calc_SSD_pool(SSD_ctrl_hom, N_hom, SSD_diet_hom, N_hom)
diff_hom = (peso_diet_hom - peso_ctrl_hom)
ES_hom = diff_hom / SSD_pool_hom

t_hom, pval_hom, msg_hom = \
calc_show_ttest_ind('homens', N_hom, lista_diet_hom, lista_ctrl_hom, ES_hom, diff_hom, SSD_pool_hom)

print(msg_hom)

In [ ]:
## grupos de mulheres independentes
lista_ctrl_mul = simular_um_grupo_ind(peso_ctrl_mul, SSD_ctrl_mul, N_mul)
lista_diet_mul = simular_um_grupo_ind(peso_diet_mul, SSD_diet_mul, N_mul)

SSD_pool_mul = calc_SSD_pool(SSD_ctrl_mul, N_mul, SSD_diet_mul, N_mul)
diff_mul = (peso_diet_mul - peso_ctrl_mul)
ES_mul = diff_mul / SSD_pool_mul

t_mul, pval_mul, msg_mul = \
calc_show_ttest_ind('mulheres', N_mul, lista_diet_mul, lista_ctrl_mul, ES_mul, diff_mul, SSD_pool_mul)

print(msg_mul)

### Teste é Bicaudal ou Monocaudal?

### Homens emagreceram mais que mulheres?